# Timing Attacks
This is a simple demonstration of a timing attack, which is a hacking method to brute force a password that takes advantage of naive string comparison operations.

## Server-side
The server holds the usernames and passwords in a "database," or just a Python Dict in this case. It also checks to see if a user entered login credentials correctly by running a string comparison on the attempted password and the actual password.

In [245]:
database = {
    "benjamin051000": "ilovejonasbros",  # Never store plaintext passwords in a db like this!
    "maisy": "meowmeow123",
    "donald": "lol",
}


def check_pass(username: str, psswd_attempt: str) -> bool:
    """Check if the entered password is correct for the user, using a naive string comparison."""
    actual_psswd = database[username]
    # return psswd_attempt == actual_psswd # This doesn't appear to work, maybe py does it differently.
    if len(psswd_attempt) != len(actual_psswd):
        return False

    for g, a in zip(psswd_attempt, actual_psswd):
        if g != a:
            return False

    return True


# Prove it works
assert check_pass("benjamin051000", "guesspassword") == False
assert check_pass("benjamin051000", "ilovejonasbros") == True

assert check_pass("maisy", "meow") == False
assert check_pass("maisy", "meowmeow123") == True


## Client-side

Assuming the string comparison algorithm first checks if the string lengths are equal, it may short-circuit and return false quickly if they are different. This can be exploited.


# 1. Crack Password Length
First, attempt to crack the length, using the notion that the "server" will respond faster if the lengths of the passwords are different.

In [246]:
import timeit
from string import ascii_letters, digits, punctuation
from random import choices


valid_chars = ascii_letters + digits  # No punctuation for now 

def random_str(length):
    return "".join(choices(valid_chars, k=length))


def crack_length(username: str, max_psswd_len=32):
    trial_results = {}

    for i in range(1, max_psswd_len + 1):
        psswd_attempt = random_str(i)

        response_times = timeit.repeat(
            stmt=f"check_pass({username!r}, {psswd_attempt!r})",
            globals=globals(),  # For check_pass()
            number=1000,  # Runs stmt this many times, enlarges differences between trials
            repeat=10,  # Run test 10 times, yield list of 10 results
        )

        fastest = min(response_times)  # Take fastest since this is the best-case

        trial_results[i] = fastest

    # Most likely length is the result with the longest time (since it passed the length check)
    return max(trial_results, key=trial_results.__getitem__)


# Prove function is able to get the length
for user in database.keys():
    assert len(database[user]) == crack_length(user)


In [247]:
# How often does it get it right?
def calc_accuracy(username, n):
    times_correct = 0
    for _ in range(n):
        most_likely = crack_length(username)
        # Was it correct?
        if most_likely == len(database[username]):
            # Add to average
            times_correct += 1

    # Calculate percent accuracy
    return times_correct / n


print(f"{calc_accuracy('benjamin051000', 100):.2%}") 


100.00%


# 2. Crack Password
Time to try to crack the actual password. In this `check_pass()` implementation, the more correct the password is, the longer `check_pass()` will take to respond.

In [248]:
import itertools

def crack_psswd(user, length):
    # Start with a random string.
    curr_guess = random_str(length)
    counter = itertools.count()

    while True:
        i = next(counter) % length  # Next index to change
        for c in valid_chars:
            # Construct alternate guess
            alt_guess = curr_guess[:i] + c + curr_guess[i+1:]

            alt_time = timeit.repeat(
                stmt="check_pass(user, alt)",
                setup=f"user={user!r}; alt={alt_guess!r}",
                globals=globals(),
                number=1000,
                repeat=10
            )

            curr_guess_time = timeit.repeat(
                stmt="check_pass(user, p)",
                setup=f"user={user!r}; p={curr_guess!r}",
                globals=globals(),
                number=1000,
                repeat=10
            )

            # If alt guess was slower, then it's more correct.
            if min(alt_time) > min(curr_guess_time):
                curr_guess = alt_guess
                print(curr_guess)

            if check_pass(user, alt_guess):
                return alt_guess
            
user = "donald"
length = crack_length(user)
print(f"{length=}")
crack_psswd(user, length)
    
    

length=3
aDz
cDz
dDz
eDz
fDz
gDz
iDz
jDz
lDz
ldz
lez
lfz
lgz
llz
loz
lol


'lol'

In [249]:
# Prove it works for all of them
# Note: This can run for several minutes if it randomly backtracks.
for user in database.keys():
    print(user)
    length = crack_length(user)
    print("Password length:", length)
    cracked_psswd = crack_psswd(user, length)
    print("Cracked psswd:", cracked_psswd)

    assert cracked_psswd == database[user]


benjamin051000
Password length: 14
alf7cPPxzazNO7
blf7cPPxzazNO7
elf7cPPxzazNO7
hlf7cPPxzazNO7
ilf7cPPxzazNO7
ilb7cPPxzazNO7
ile7cPPxzazNO7
ilf7cPPxzazNO7
ili7cPPxzazNO7
ilj7cPPxzazNO7
ilk7cPPxzazNO7
ilm7cPPxzazNO7
iln7cPPxzazNO7
ilo7cPPxzazNO7
iloacPPxzazNO7
iloccPPxzazNO7
ilodcPPxzazNO7
iloecPPxzazNO7
ilomcPPxzazNO7
iloncPPxzazNO7
ilorcPPxzazNO7
ilotcPPxzazNO7
ilovcPPxzazNO7
ilovaPPxzazNO7
ilovbPPxzazNO7
ilovcPPxzazNO7
ilovePPxzazNO7
iloveaPxzazNO7
ilovecPxzazNO7
iloveePxzazNO7
ilovejPxzazNO7
ilovejaxzazNO7
ilovejbxzazNO7
ilovejcxzazNO7
ilovejdxzazNO7
ilovejfxzazNO7
ilovejgxzazNO7
ilovejhxzazNO7
ilovejjxzazNO7
ilovejoxzazNO7
ilovejobzazNO7
ilovejodzazNO7
ilovejoezazNO7
ilovejohzazNO7
ilovejoizazNO7
ilovejojzazNO7
ilovejokzazNO7
ilovejonzazNO7
ilovejonaazNO7
ilovejonaazNO7
ilovejonaizNO7
ilovejonajzNO7
ilovejonalzNO7
ilovejonamzNO7
ilovejonapzNO7
ilovejonaqzNO7
ilovejonarzNO7
ilovejonaszNO7
ilovejona1zNO7
ilovejona2zNO7
ilovejona3zNO7
ilovejona4zNO7
ilovejona5zNO7
ilovejona7zNO7
ilove

Long story short, don't use this type of string comparison! Instead, use a constant-time string comparison, e.g., `hmac.compare_digest()`.